In [2]:
import tensorflow as tf
from ray.rllib.offline.json_reader import JsonReader
import numpy as np
import numpy_indexed as npi
import pandas as pd
from true_state_viewer import TrueStateTreeGraphViz, display_tree_pairs
from tqdm import trange
import os

/usr/local/lib/python3.8/dist-packages/tensorflow_probability/python/__init__.py:57: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if (distutils.version.LooseVersion(tf.__version__) <


In [19]:
path = 'logs/APPO/TrueStates_200_1000_Meander_small'
input_reader = JsonReader(path)
num_episodes = int(200*10)
num_data_points = num_episodes * 99
state_length = 42
num_actions = 20

states = np.ones((num_data_points, state_length))
states_t = np.ones((num_data_points, state_length+1))
afterstates = np.ones((num_data_points, state_length))
actions_onehot = np.zeros((num_data_points, num_actions))
rewards = np.ones(num_data_points) 
next_states = np.ones((num_data_points, state_length))
inital_states = []

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    states[index:index+99,:] = data['obs'][0:99]
    states_t[index:index+99,:] = np.concatenate([data['obs'][0:99],np.arange(1,100).reshape(99,1)/100], axis=1)
    next_states[index:index+99,:] = data['obs'][1:100]
    afterstates[index:index+99,:] = data['afterstates'][0:99]
    rewards[index:index+99] = data['rewards'][0:99]
    actions_onehot[np.arange(index,index+99),data['actions'][0:99]] = 1  
    index += 99
    
    if not data['obs'][0].tobytes() in inital_states:
        inital_states.append(data['obs'][0].tobytes())

2023-02-08 12:46:58,600	WARNING json_reader.py:239 -- Treating input directory as glob patterns: ['/tf/Cage/Notebooks/logs/APPO/TrueStates_200_1000_Meander_small/*.json', '/tf/Cage/Notebooks/logs/APPO/TrueStates_200_1000_Meander_small/*.zip']
100%|██████████| 2000/2000 [00:39<00:00, 51.05it/s]


In [21]:
#os.mkdir(path + '/data')
np.save(path + '/data/states.npy', states)
np.save(path + '/data/states_t.npy', states_t)
np.save(path + '/data/next_states.npy', next_states)
np.save(path + '/data/afterstates.npy', afterstates)
np.save(path + '/data/rewards.npy', rewards)
np.save(path + '/data/actions_onehot.npy', actions_onehot)

In [60]:
episode_len = 99

states_seq = np.zeros((num_data_points, episode_len, state_length))
actions_onehot_seq = np.zeros((num_data_points,episode_len, num_actions))

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    for t in range(1,episode_len):
        states_seq[index+t,0:t,:] = data['obs'][0:t]
        actions_onehot_seq[index+t,0:t,data['actions'][0:t]] = 1  
    index += 99

100%|██████████| 2000/2000 [00:45<00:00, 43.69it/s]


In [61]:
np.save(path + '/data/states_seq.npy', states_seq)
np.save(path + '/data/actions_onehot_seq.npy', actions_onehot_seq)

In [66]:
episode_len = 99
window = 20

states_seq = np.zeros((num_data_points, window, state_length))
actions_onehot_seq = np.zeros((num_data_points,window, num_actions))

index = 0
for e in trange(num_episodes):
    data = input_reader.next()
    for t in range(1,episode_len-window):
        states_seq[index+t,:,:] = data['obs'][t:t+window]
        actions_onehot_seq[index+t,:,data['actions'][t:t+window]] = 1  
    index += 99

100%|██████████| 2000/2000 [00:39<00:00, 50.18it/s]


In [67]:
np.save(path + '/data/states_seq_20.npy', states_seq)
np.save(path + '/data/actions_onehot_seq_20.npy', actions_onehot_seq)